In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from typing import List

import openai
import os
import sys
from dotenv import load_dotenv

#OPENAI_API_KEY = "sk-proj-AO3KelC1emoiTrtYed3jT3BlbkFJVuDMTmSkpm2J4S7me2nO"
#load_dotenv()    
#openai.api_key = os.getenv(OPENAI_API_KEY)

from openai import OpenAI
OPENAI_API_KEY = "sk-proj-AO3KelC1emoiTrtYed3jT3BlbkFJVuDMTmSkpm2J4S7me2nO"
client = OpenAI(
  api_key=OPENAI_API_KEY 
)

In [2]:
from urllib import request
from urllib.request import Request, urlopen
from PIL import Image
import pytesseract
from io import BytesIO

In [4]:
from ast import literal_eval
import re
from tqdm.notebook import tqdm

import konlpy
from konlpy.tag import Okt

In [5]:
import torch
from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer, util
import json
import uuid

import requests
import urllib3
import random

/var/folders/qw/4pc1z8g93n746cysqbq8tq840000gn/T/ipykernel_5131/3617152277.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### 1. 데이터 준비

In [5]:
data = pd.read_excel('/Users/parkjungeun/DML_crawling/KB_공모전/kb_event_total.xlsx')
#print(data.shape)

#전처리
data['img'] = data['img'].apply(lambda x: literal_eval(x))
data['알아두세요'] = data['알아두세요'].apply(lambda x : re.sub("(알아두세요|\n)",'',x))

In [6]:
#NAVER CLOVA OCR 수행
API_URL  = 'https://eqot8fiwsu.apigw.ntruss.com/custom/v1/33378/e50923c86e03947b413aedacc666a38095e81ea1cf542b3872634c081f839fbd/general'
API_SECRET_KEY = 'V1FnTEVTUFV3ZFl3eURrS0J3T3hRSGlIblVKd2ROb20='
 
#SSL 경고 무시
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def ocr_clova(image_url):
    """URL에서 이미지를 읽고 클로바 OCR API에 요청하는 함수"""

    # URL에서 이미지 다운로드
    response = requests.get(image_url, verify=False)  # SSL 검증 비활성화
    image_data = BytesIO(response.content)

    headers = {
        'X-OCR-SECRET': API_SECRET_KEY,
    }

    files = {
        'file': image_data
    }

    # 요청 바디 구성
    data = {
        'version': 'V2',
        'requestId': 'sample_id',  # 요청을 구별하기 위한 임의의 ID
        'timestamp': 0,  # 현재 시간 (필요한 경우 시간을 입력)
        'images': [
            {
                'format': 'jpg',
                'name': 'sample_image',
            }
        ]
    }

    # API 요청 보내기
    response = requests.post(API_URL, headers=headers, data={'message': json.dumps(data)}, files=files, verify=False)  # SSL 검증 비활성화
    result = response.json()

   # 응답 처리 및 문장 단위로 출력
    if response.status_code == 200:
        full_text = ""
        for image in result['images']:
            for field in image['fields']:
                full_text += field['inferText'] + " "
        
        # 문장 단위로 텍스트 분리 (기본적인 방법)
        sentences = full_text.split('. ')
        
        # 출력
        #for sentence in sentences:
            #print(sentence.strip())
    #else:
        #print(f"Error: {result}")
    return full_text

In [7]:
#이미지 -> 텍스트 변환
result = []
j = 0
for img_list in tqdm(data['img']):
    img_txt = []
    for i in img_list:
        try:
            text = ocr_clova(i)
            img_txt.append(text)
        except:
            img_txt.append(' ') #img url 에러뜨면 넘어가기
    j += 1
    print('#------'+str(j)+'------#')
    print(' '.join(img_txt))
    result.append(' '.join(img_txt))

result

  0%|          | 0/690 [00:00<?, ?it/s]

#------1------#
+ [3회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하세요  이벤트 대상 - 각 이벤트 회차 내 아래 1. 2조건을 모두 충족한 고객 중 추첨을 통해 커피쿠폰을 드려요 1 KB스타(인터넷)뱅킹에서 기타통화를 미화 100불 상당액 이상 원화 → 외화 환전 신청 2 KB스타뱅킹 이벤트 페이지에서 응모 * 환전과 응모의 순서는 무관합니다. - 기타통화 USD, JPY, EUR을 제외한 비대면 신청가능 통화 14종 CNY.THB.GBP.AD, CHF, HKD,AUD,SGDNZD. VND. PHP. TWD. MYR.IDR 이벤트 기간 2024.7.15(월) ~ 2024.8.18(일) [약 5주간] 이벤트 내용 경품 : 스타벅스 아이스 카페 아메리카노 T 2잔 교환권 [이벤트 일정] 회차 이벤트 참여기간 당첨자수 당첨 발표일 경품 발송일 1차 '24.7.15(월) ~ 7.21(일) 200명 '24.7.23(화) '24.7.25(목) 2차 '24.7.22(월) ~ 7.28(일) 200명 '24.7.30(화) '24.8.1(목) 3차 '24.7.29(월) ~8.4(일) 200명 '24.8.6(화) '24.8.8(목) 4차 '24.8.5(월) ~8.11(일) 200명 '24.8.13(화) '24.8.14(목) 5차 '24.8.12(월) ~ 8.18(일) 200명 '24.8.20(화) '24.8.22(목) 이벤트 대상 비대면 환전서비스 1) KB외화머니박스 입금 2) 공항에서 받기 3) 지점에서 받기 4) 외화배달서비스 5) 외화선물하기(선물하는 고객만 응모 가능) 비로그인 환전(가상계좌환전)은 제외 
#------2------#
+ [2회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하세요  이벤트 대상 - 각 이벤트 회차 내 아래 1. 2조건을 모두 충족한 고객 중 추첨을 통해 커피쿠폰을 드려요 1 KB스타(인터넷)뱅킹에서 기타통화를 미화 100불 상당액 이상 원화 → 외화 환전 신청 2 KB

["+ [3회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하세요  이벤트 대상 - 각 이벤트 회차 내 아래 1. 2조건을 모두 충족한 고객 중 추첨을 통해 커피쿠폰을 드려요 1 KB스타(인터넷)뱅킹에서 기타통화를 미화 100불 상당액 이상 원화 → 외화 환전 신청 2 KB스타뱅킹 이벤트 페이지에서 응모 * 환전과 응모의 순서는 무관합니다. - 기타통화 USD, JPY, EUR을 제외한 비대면 신청가능 통화 14종 CNY.THB.GBP.AD, CHF, HKD,AUD,SGDNZD. VND. PHP. TWD. MYR.IDR 이벤트 기간 2024.7.15(월) ~ 2024.8.18(일) [약 5주간] 이벤트 내용 경품 : 스타벅스 아이스 카페 아메리카노 T 2잔 교환권 [이벤트 일정] 회차 이벤트 참여기간 당첨자수 당첨 발표일 경품 발송일 1차 '24.7.15(월) ~ 7.21(일) 200명 '24.7.23(화) '24.7.25(목) 2차 '24.7.22(월) ~ 7.28(일) 200명 '24.7.30(화) '24.8.1(목) 3차 '24.7.29(월) ~8.4(일) 200명 '24.8.6(화) '24.8.8(목) 4차 '24.8.5(월) ~8.11(일) 200명 '24.8.13(화) '24.8.14(목) 5차 '24.8.12(월) ~ 8.18(일) 200명 '24.8.20(화) '24.8.22(목) 이벤트 대상 비대면 환전서비스 1) KB외화머니박스 입금 2) 공항에서 받기 3) 지점에서 받기 4) 외화배달서비스 5) 외화선물하기(선물하는 고객만 응모 가능) 비로그인 환전(가상계좌환전)은 제외 ",
 "+ [2회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하세요  이벤트 대상 - 각 이벤트 회차 내 아래 1. 2조건을 모두 충족한 고객 중 추첨을 통해 커피쿠폰을 드려요 1 KB스타(인터넷)뱅킹에서 기타통화를 미화 100불 상당액 이상 원화 → 외화 환전 신청 2 KB스타뱅킹 이벤트 페이지에서 응모 * 환전과 응모

In [12]:
pd.DataFrame(data = result, columns = ['img_text'])

,img_text
0,+ [3회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하...
1,+ [2회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하...
2,KB국민은행 X 와이파이도시락 걸어서 할인속으로 와이파이도시락 20% 할인 PASS...
3,+ [1회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하...
4,랜덤 포인트가 왔썸머~ KB스타뱅킹 KB Pay 결제 이벤트! KB Pay 기간 ...
...,...
685,신상품이 받고 싶은 나왔네여? 경품은 내가 골라요! KB마이핏통장 # KB마이핏적금...
686,"KB가 Liiv M을 만날 때 통신비 0원, 반갑다! Liiv M 첫 달 무료, 1..."
687,+ NA KB *b + + 국민은행 M *b 이벤트 기간 2020.6.1.~202...
688,이벤트 기간 2020.5.25(월) ~2020. 6. 30(화) 대상상품(1종) ...


In [9]:
#pd.DataFrame(data = result, columns = ['img_text']).to_excel('/Users/parkjungeun/Downloads/KB/img_text_total.xlsx', index = False)
#pd.DataFrame(data = result, columns = ['img_text']).to_csv('/Users/parkjungeun/Downloads/KB/img_text_total.csv', index = False)

In [31]:
img_text = pd.DataFrame(data = result, columns = ['img_text'])
img_text = img_text.reset_index()
#data = data.reset_index()
df = pd.concat([data, img_text], axis=1)
df = df.iloc[:,~(df.columns.isin(['level_0','index']))]
df.head(2)

,번호,제목,이벤트 기간,조회수,img,알아두세요,img_text
0,1,[3회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하세요...,2024.07.29 ~ 2024.08.04,334,[https://oimg1.kbstar.com/img/oevent/2024/m07/...,"전체 이벤트 기간 중 1회만 당첨 가능하며, 이전 회차에 당첨된 이력이 있는 고객님...",+ [3회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하...
1,2,[2주차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하세요♥,2024.07.22 ~ 2024.07.28,374,[https://oimg1.kbstar.com/img/oevent/2024/m07/...,"전체 이벤트 기간 중 1회만 당첨 가능하며, 이전 회차에 당첨된 이력이 있는 고객님...",+ [2회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하...


In [32]:
df.to_excel('/Users/parkjungeun/Downloads/KB/df_total.xlsx', index = False)
df.to_csv('/Users/parkjungeun/Downloads/KB/df_total.csv', index = False)

In [41]:
img_text.isna().sum()

index       0
img_text    0
dtype: int64

### 2. Embedding vector 추출 : OpenAI Embedding 활용

In [125]:
df = pd.read_csv('/Users/parkjungeun/Downloads/KB/df_total.csv')
df.head(2)

,번호,제목,이벤트 기간,조회수,img,알아두세요,img_text
0,1,[3회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하세요...,2024.07.29 ~ 2024.08.04,334,['https://oimg1.kbstar.com/img/oevent/2024/m07...,"전체 이벤트 기간 중 1회만 당첨 가능하며, 이전 회차에 당첨된 이력이 있는 고객님...",+ [3회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하...
1,2,[2주차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하세요♥,2024.07.22 ~ 2024.07.28,374,['https://oimg1.kbstar.com/img/oevent/2024/m07...,"전체 이벤트 기간 중 1회만 당첨 가능하며, 이전 회차에 당첨된 이력이 있는 고객님...",+ [2회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하...


In [127]:
df['img_text'][0]

"+ [3회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하세요  이벤트 대상 - 각 이벤트 회차 내 아래 1. 2조건을 모두 충족한 고객 중 추첨을 통해 커피쿠폰을 드려요 1 KB스타(인터넷)뱅킹에서 기타통화를 미화 100불 상당액 이상 원화 → 외화 환전 신청 2 KB스타뱅킹 이벤트 페이지에서 응모 * 환전과 응모의 순서는 무관합니다. - 기타통화 USD, JPY, EUR을 제외한 비대면 신청가능 통화 14종 CNY.THB.GBP.AD, CHF, HKD,AUD,SGDNZD. VND. PHP. TWD. MYR.IDR 이벤트 기간 2024.7.15(월) ~ 2024.8.18(일) [약 5주간] 이벤트 내용 경품 : 스타벅스 아이스 카페 아메리카노 T 2잔 교환권 [이벤트 일정] 회차 이벤트 참여기간 당첨자수 당첨 발표일 경품 발송일 1차 '24.7.15(월) ~ 7.21(일) 200명 '24.7.23(화) '24.7.25(목) 2차 '24.7.22(월) ~ 7.28(일) 200명 '24.7.30(화) '24.8.1(목) 3차 '24.7.29(월) ~8.4(일) 200명 '24.8.6(화) '24.8.8(목) 4차 '24.8.5(월) ~8.11(일) 200명 '24.8.13(화) '24.8.14(목) 5차 '24.8.12(월) ~ 8.18(일) 200명 '24.8.20(화) '24.8.22(목) 이벤트 대상 비대면 환전서비스 1) KB외화머니박스 입금 2) 공항에서 받기 3) 지점에서 받기 4) 외화배달서비스 5) 외화선물하기(선물하는 고객만 응모 가능) 비로그인 환전(가상계좌환전)은 제외 "

In [124]:
df.head(3)

,번호,제목,이벤트 기간,조회수,img,알아두세요,img_text
0,1,[3회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하세요...,2024.07.29 ~ 2024.08.04,334,['https://oimg1.kbstar.com/img/oevent/2024/m07...,"전체 이벤트 기간 중 1회만 당첨 가능하며, 이전 회차에 당첨된 이력이 있는 고객님...",+ [3회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하...
1,2,[2주차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하세요♥,2024.07.22 ~ 2024.07.28,374,['https://oimg1.kbstar.com/img/oevent/2024/m07...,"전체 이벤트 기간 중 1회만 당첨 가능하며, 이전 회차에 당첨된 이력이 있는 고객님...",+ [2회차] HOT SUMMER 환전 이벤트! 환전하고 COOL하게 커피 한잔 하...
2,3,[KB국민은행X와이파이도시락] 걸어서 할인속으로 와이파이도시락 2O% 할인 | 진행중,2024.07.17 ~ 2024.09.13,425,['https://oimg1.kbstar.com/img/oevent/2024/m07...,본 이벤트는 KB스타뱅킹 App(최종 업데이트 버전)의 이벤트 페이지에서만 1인 1...,KB국민은행 X 와이파이도시락 걸어서 할인속으로 와이파이도시락 20% 할인 PASS...


In [7]:
df['feature'] = df['제목'] #+ " / " + df['이벤트 기간'] + " / " + df['img_text']

In [8]:
df.drop(labels = df[df['feature'].isna()].index, errors='ignore', inplace = True) #img_text 빈 행 제거
df.reset_index(inplace = True)

In [9]:
openai_model = 'text-embedding-ada-002'
overview_emb = []
for i in tqdm(range(df.shape[0])):
    text = df['feature'][i]
    result = client.embeddings.create(model = openai_model, input = text)
    result_emb = result.data[0].embedding
    overview_emb.append(result_emb)

  0%|          | 0/690 [00:00<?, ?it/s]

In [10]:
df['openai_emb'] = overview_emb
df = df.loc[:, ~(df.columns.isin(['level_0','index']))]
#print(df.columns)
#print(df.shape)
#df.head(2)

In [11]:
len(df['openai_emb'][0]) #OpenAI의 embedding차원은 1536

1536

In [47]:
#df.to_csv('/Users/parkjungeun/Downloads/KB/df_emb_total_2.csv',index = False)

### 3. 코사인 유사도 구현

In [128]:
df = pd.read_csv('/Users/parkjungeun/Downloads/KB/df_emb_total_2.csv', sep = ',', dtype = str)
df['openai_emb'] = df['openai_emb'].apply(lambda x: list(literal_eval(x)))
df['openai_emb'] = df['openai_emb'].apply(lambda x: list(x))

In [19]:
def get_query_sim_top_k(query, top_k):
    openai_model = 'text-embedding-ada-002'
    query_encode = client.embeddings.create(model = openai_model, input=query)
    query_encode = query_encode.data[0].embedding
    cos_scores = util.pytorch_cos_sim(query_encode, df['openai_emb'])[0]
    top_results = torch.topk(cos_scores, k=top_k)
    print(top_results)

    top_result_id = top_results.indices.tolist()
    top_result_score = top_results.values.tolist()

    result_df = df[df.index.isin(top_result_id)]
    result_df.index = top_result_id
    result_df['score'] = top_result_score
    answer = result_df[['score','번호','제목', '이벤트 기간', 'img', 'img_text']]

    return top_results #answer

In [20]:
query = '스타벅스 커피와 관련된 이벤트'
top_result = get_query_sim_top_k(query, top_k = 5)
df.iloc[top_result[1].numpy(), :][['번호','제목', '이벤트 기간', 'img','img_text']]
#get_query_sim_top_k(query = '스타벅스 커피와 관련된 이벤트', top_k = 5)

torch.return_types.topk(
values=tensor([0.9071, 0.8824, 0.8774, 0.8704, 0.8653]),
indices=tensor([636, 589,  70, 375, 658]))


/var/folders/qw/4pc1z8g93n746cysqbq8tq840000gn/T/ipykernel_5131/1429551103.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['score'] = top_result_score


,번호,제목,이벤트 기간,img,img_text
636,762,스타벅스 커피 5잔 공약! 「KB골든라이프」 카카오 플친 이벤트,2020.10.26 ~ 2020.11.25,['https://oimg1.kbstar.com/img/oevent/2020/102...,스타벅스 커피 5잔 공약! KB필드라탕 TALK 골든라이프 80ts - KB ###...
589,711,KB스타뱅킹에서 고객님의 목소리를 기다립니다! 1만명에게 스타벅스 커피드려요!,2021.02.16 ~ 2021.03.31,['https://oimg1.kbstar.com/img/oevent/2021/021...,KB스타뱅킹에서 고객님의 목소리를 기다립니다! #1만명에게 스타벅스 커피드려요! *...
70,83,캠퍼스스타 20기 모집 이벤트,2024.01.04 ~ 2024.01.28,['https://oimg1.kbstar.com/img/oevent/2024/m01...,"찾았다, 내 스무번째 별 KB국민은행과 함께하는 KB 캠퍼스스타 20기 지원서 작..."
375,452,KB스타뱅킹 고객 대환영 이벤트!,2022.04.05 ~ 2022.04.29,['https://oimg1.kbstar.com/img/oevent/2022/040...,KB스타뱅킹 고객 대환영 이벤트! *b KB국민은행 KB스타뱅킹에서 세라젬 당첨 기...
658,784,KB스타뱅킹X오픈뱅킹 경험하면 내가 원하는 스타급 경품을 내손에!! 보너스 커피쿠폰...,2020.09.01 ~ 2020.10.16,['https://oimg1.kbstar.com/img/oevent/2020/090...,오픈뱅킹 최초 등록하고 내가 원하는 스타급 경품을 직접 고르세요! KB 국민은행 응...


In [21]:
query = '대출'
top_result = get_query_sim_top_k(query, top_k = 5)
df.iloc[top_result[1].numpy(), :][['번호','제목', '이벤트 기간', 'img','img_text']]

torch.return_types.topk(
values=tensor([0.8674, 0.8564, 0.8526, 0.8492, 0.8463]),
indices=tensor([355, 340, 408,  38, 687]))


/var/folders/qw/4pc1z8g93n746cysqbq8tq840000gn/T/ipykernel_5131/1429551103.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['score'] = top_result_score


,번호,제목,이벤트 기간,img,img_text
355,431,비대면 전월세보증금 대출 이벤트,2022.05.02 ~ 2022.05.19,['https://oimg1.kbstar.com/img/oevent/2022/050...,전월세보증금 비대면 대출 이벤트 CU # 첫 달 이자 지원금 100% # 맞춤 모바...
340,416,비대면 전월세보증금 대출 이벤트 PART2,2022.05.20 ~ 2022.06.30,['https://oimg1.kbstar.com/img/oevent/2022/052...,"비대면 전월세보증금 대출 이벤트 PART2. 비대면으로 전월세보증금대출 받으면, 이..."
408,488,비대면 「주택자금대출」 (전세/담보) 통합 이벤트,2022.01.28 ~ 2022.04.29,['https://oimg1.kbstar.com/img/oevent/2022/011...,KB국민은행이 2022년 새출발을 응원합니다 비대면 주택자금대출 전세·담보 통합 이...
38,41,[소상공인 응원 프로젝트 Ⅰ]소상공인 보증서대출 보증료 환급 | 진행중,2024.04.03 ~ 2024.09.25,['https://oimg1.kbstar.com/img/oevent/2024/040...,소상공인 응원 프로젝트I 소상공인 보증서대출 80% 보증료 환급 70% 60% 최대...
687,828,신용대출도 역시 KB이지(Easy)~,2020.06.01 ~ 2020.08.31,['https://oimg1.kbstar.com/img/oevent/2020/060...,+ NA KB *b + + 국민은행 M *b 이벤트 기간 2020.6.1.~202...


### 4. 고객 정보 데이터 전처리

In [75]:
df1= pd.read_csv(r'/Users/parkjungeun/Downloads/KB/서울 시민의 금융 생활 재현 데이터.csv',encoding='utf-8')
df2= pd.read_csv(r'/Users/parkjungeun/Downloads/KB/서울 시민의 소비 및 이동 패턴 재현 데이터.csv',encoding='utf-8')

# 금융 생활
df1=df1[['자치구명','생년월일','성별','직업군','대출여부','사업자여부','주택보유건수','LTV','온라인_카드사용금액_백분위수','오프라인_카드사용금액_백분위수','사업자카드이용액']]
# 소비 및 이동패턴
df2=df2[['자치구명','생년월일','성별','직업군','평균_통화량','대출여부','사업자여부','1인가구여부','가맹점분류_이용비중1','가맹점분류_이용비중2','가맹점분류_이용비중3','가맹점분류_이용비중4','가맹점분류_이용비중5','가맹점분류_이용비중6','가맹점분류_이용비중7','가맹점분류_이용비중8','가맹점분류_이용비중9','가맹점분류_이용비중10','가맹점분류_이용비중11','가맹점분류_이용비중12','가맹점분류_이용비중13','가맹점분류_이용비중14','가맹점분류_이용비중15','가맹점분류_이용비중16','가맹점분류_이용비중17','가맹점분류_이용비중18','가맹점분류_이용비중19','해외카드이용금액','추정연소득_백분위']]

df1['LTV']=df1['LTV']*100
df1['자치구명']='서울시 '+df1['자치구명']
df2['자치구명']='서울시 '+df2['자치구명']
df1.rename(columns={'자치구명':'거주지역','LTV':'LTV(%)'},inplace=True)

display(df1.head(3))
display(df2.head(3))

,거주지역,생년월일,성별,직업군,대출여부,사업자여부,주택보유건수,LTV(%),온라인_카드사용금액_백분위수,오프라인_카드사용금액_백분위수,사업자카드이용액
0,서울시 관악구,2002,여,910,0,0,0,0.0,NaN,NaN,0
1,서울시 양천구,1976,여,910,0,0,0,0.0,NaN,NaN,0
2,서울시 마포구,1997,남,910,0,0,0,0.0,NaN,NaN,0


,자치구명,생년월일,성별,직업군,평균_통화량,대출여부,사업자여부,1인가구여부,가맹점분류_이용비중1,가맹점분류_이용비중2,...,가맹점분류_이용비중12,가맹점분류_이용비중13,가맹점분류_이용비중14,가맹점분류_이용비중15,가맹점분류_이용비중16,가맹점분류_이용비중17,가맹점분류_이용비중18,가맹점분류_이용비중19,해외카드이용금액,추정연소득_백분위
0,서울시 강동구,1978,남,420,NaN,0,0,0,0.158095,0.0,...,0.0,0.0,0.006462,0.0,0.0,0.032664,0.140745,0.531734,87,10.0
1,서울시 도봉구,1943,남,420,NaN,0,0,0,0.532911,0.0,...,0.0,0.0,0.201899,0.0,0.0,0.000000,0.000000,0.000000,0,64.0
2,서울시 동작구,1956,남,420,NaN,1,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71.0


In [76]:
name=str('''가맹점분류_이용비중1 외식/카페_이용비중
가맹점분류_이용비중2 유흥_이용비중
가맹점분류_이용비중3 백화점_이용비중
가맹점분류_이용비중4 대형할인점_이용비중
가맹점분류_이용비중5 소형유통점_이용비중
가맹점분류_이용비중6 의류/잡화_이용비중
가맹점분류_이용비중7 스포츠/문화/레저_이용비중
가맹점분류_이용비중8 숙박_이용비중
가맹점분류_이용비중9 여행_이용비중
가맹점분류_이용비중10 교통_이용비중
가맹점분류_이용비중11 미용_이용비중
가맹점분류_이용비중12 가정생활/서비스_이용비중
가맹점분류_이용비중13 교육/학원_이용비중
가맹점분류_이용비중14 의료_이용비중
가맹점분류_이용비중15 가전/가구_이용비중
가맹점분류_이용비중16 자동차판매_이용비중
가맹점분류_이용비중17 자동차서비스/용품_이용비중
가맹점분류_이용비중18 주유_이용비중
가맹점분류_이용비중19 전자상거래_이용비중''')

line = name.strip().split('\n')
name_dict={}
for i in line:
    key,value =i.split()
    name_dict[key] = value.replace('_이용비중','')

In [78]:
df2.rename(columns=name_dict,inplace=True)

In [80]:
# 각 행에서 상위 3개 부문을 선택하는 함수
def get_top_3_categories(row):
    # 상위 3개 값 선택
    top_3 = row.nlargest(3)
    return top_3.index.tolist()

# 각 행에 대해 상위 3개 부문 선택
df2['Top3_소비_키워드'] = df2.iloc[:,8:26].apply(get_top_3_categories, axis=1)

#행 삭제
df2.drop(df2.columns[7:26],axis=1,inplace=True)

In [82]:
## 직업 인코딩 변환 
### 대기업 급여 소득 -> 대기업 직원
### 일반 급여 소득 -> 일반 직장인
### 전문직 급여 소득 -> 전문직 종사

In [83]:
job_list=str('''410(대기업직원),420(일반직장인),430(전문직), 440(대표),510(일반자영업), 520(전문직자영업),910(무직/기타)''')
line= job_list.strip().split(',')
job_dict={}
for i in line:
    key,value =i.split('(')
    job_dict[int(key)] = value.replace(')','')

In [85]:
job_dict

{410: '대기업직원',
 420: '일반직장인',
 430: '전문직',
 440: '대표',
 510: '일반자영업',
 520: '전문직자영업',
 910: '무직/기타'}

In [86]:
df2=df2.replace(job_dict)
df1=df1.replace(job_dict)

In [88]:
display(df1.head(2))
display(df2.head(2))

,거주지역,생년월일,성별,직업군,대출여부,사업자여부,주택보유건수,LTV(%),온라인_카드사용금액_백분위수,오프라인_카드사용금액_백분위수,사업자카드이용액
0,서울시 관악구,2002,여,무직/기타,0,0,0,0.0,NaN,NaN,0
1,서울시 양천구,1976,여,무직/기타,0,0,0,0.0,NaN,NaN,0


,자치구명,생년월일,성별,직업군,평균_통화량,대출여부,사업자여부,전자상거래,해외카드이용금액,추정연소득_백분위,Top3_소비_키워드
0,서울시 강동구,1978,남,일반직장인,NaN,0,0,0.531734,87,10.0,"[외식/카페, 주유, 소형유통점]"
1,서울시 도봉구,1943,남,일반직장인,NaN,0,0,0.000000,0,64.0,"[외식/카페, 의료, 의류/잡화]"


In [92]:
# 0값을 null로 치환
df1 = df1.replace(0, np.nan)
df2 = df2.replace(0, np.nan)

core=df2[['Top3_소비_키워드','직업군','생년월일','성별','대출여부']]

In [96]:
#json 변환
def save_to_json(df, file_name, start_id=0):
    # 인덱스를 id 키로 추가
    df = df.reset_index().rename(columns={'index': 'id'})
    df['id'] += start_id

    # NaN 값을 제거하고 JSON 변환
    result = []
    for _, row in df.iterrows():
        record = row.dropna().to_dict()
        result.append(record)

    with open(file_name, 'w', encoding='utf-8') as f:
      json.dump(result, f, ensure_ascii=False, indent=4)

    return print(json.dumps(result, ensure_ascii=False, indent=4))

In [97]:
#save_to_json(core, '/Users/parkjungeun/Downloads/KB/core.json', start_id=0)

[
    {
        "id": 0,
        "Top3_소비_키워드": [
            "외식/카페",
            "주유",
            "소형유통점"
        ],
        "직업군": "일반직장인",
        "생년월일": 1978,
        "성별": "남"
    },
    {
        "id": 1,
        "Top3_소비_키워드": [
            "외식/카페",
            "의료",
            "의류/잡화"
        ],
        "직업군": "일반직장인",
        "생년월일": 1943,
        "성별": "남"
    },
    {
        "id": 2,
        "Top3_소비_키워드": [
            "외식/카페",
            "유흥",
            "백화점"
        ],
        "직업군": "일반직장인",
        "생년월일": 1956,
        "성별": "남",
        "대출여부": 1.0
    },
    {
        "id": 3,
        "Top3_소비_키워드": [
            "외식/카페",
            "의류/잡화",
            "의료"
        ],
        "직업군": "일반직장인",
        "생년월일": 1979,
        "성별": "여",
        "대출여부": 1.0
    },
    {
        "id": 4,
        "Top3_소비_키워드": [
            "외식/카페",
            "의료",
            "소형유통점"
        ],
        "직업군": "일반직장인",
        "생년월일": 1941,
        "성별": "여"
    },
    {
       

In [68]:
with open('/Users/parkjungeun/Downloads/KB/total_output.json', 'r', encoding='utf-8') as f:
    customer_data = json.load(f)

# JSON 데이터를 id를 키로 사용하는 딕셔너리로 변환
customer_dict = {customer['id']: customer for customer in customer_data}
#customer_dict

{0: {'id': 0, '거주지역': '서울시 관악구', '생년월일': 2002, '성별': '여', '직업군': '무직/기타'},
 1: {'id': 1, '거주지역': '서울시 양천구', '생년월일': 1976, '성별': '여', '직업군': '무직/기타'},
 2: {'id': 2, '거주지역': '서울시 마포구', '생년월일': 1997, '성별': '남', '직업군': '무직/기타'},
 3: {'id': 3,
  '거주지역': '서울시 강북구',
  '생년월일': 1977,
  '성별': '여',
  '직업군': '무직/기타',
  '온라인_카드사용금액_백분위수': 61.0,
  '오프라인_카드사용금액_백분위수': 100.0},
 4: {'id': 4,
  '거주지역': '서울시 강남구',
  '생년월일': 2002,
  '성별': '남',
  '직업군': '무직/기타',
  '온라인_카드사용금액_백분위수': 100.0,
  '오프라인_카드사용금액_백분위수': 100.0},
 5: {'id': 5, '거주지역': '서울시 금천구', '생년월일': 2002, '성별': '남', '직업군': '무직/기타'},
 6: {'id': 6,
  '거주지역': '서울시 중랑구',
  '생년월일': 1999,
  '성별': '남',
  '직업군': '무직/기타',
  '대출여부': 1.0,
  '온라인_카드사용금액_백분위수': 100.0,
  '오프라인_카드사용금액_백분위수': 100.0},
 7: {'id': 7, '거주지역': '서울시 중구', '생년월일': 1941, '성별': '여', '직업군': '무직/기타'},
 8: {'id': 8, '거주지역': '서울시 은평구', '생년월일': 1972, '성별': '여', '직업군': '무직/기타'},
 9: {'id': 9,
  '거주지역': '서울시 강남구',
  '생년월일': 1967,
  '성별': '남',
  '직업군': '일반직장인',
  '온라인_카드사용금액_백분위수': 7.0,
  '오프라인_카

### 5. Chatgpt 기반 추천시스템

In [22]:
with open('/Users/parkjungeun/Downloads/KB/core.json', 'r', encoding='utf-8') as f:
    customer_data = json.load(f)

# JSON 데이터를 id를 키로 사용하는 딕셔너리로 변환
customer_dict = {customer['id']: customer for customer in customer_data}

In [55]:
def get_query_sim_top_k(query, top_k):
    openai_model = 'text-embedding-ada-002'
    query_encode = client.embeddings.create(model = openai_model, input=query)
    query_encode = query_encode.data[0].embedding
    cos_scores = util.pytorch_cos_sim(query_encode, df['openai_emb'])[0]
    top_results = torch.topk(cos_scores, k=top_k)
    print(top_results)

    top_result_id = top_results.indices.tolist()
    top_result_score = top_results.values.tolist()

    result_df = df[df.index.isin(top_result_id)]
    result_df.index = top_result_id
    result_df['score'] = top_result_score
    answer = result_df[['score','번호','제목', '이벤트 기간', 'img', 'img_text']]

    return top_results #answer

#user_msg_history = []
def get_chatgpt_msg(msg):
    response = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=msg
                    )
    return response.choices[0].message.content

def set_prompt(intent, query, msg_prompt_init):
    '''prompt 형태를 만들어주는 함수'''
    m = dict()
    # 검색 또는 추천이면
    if ('recom' in intent) or ('search' in intent):
        msg = msg_prompt_init['recom'] # 시스템 메세지를 가지고오고
    # 설명문이면
    elif 'desc' in intent:
        msg = msg_prompt_init['desc'] # 시스템 메세지를 가지고오고
    # intent 파악
    else:
        msg = msg_prompt_init['intent']
        msg['user'] += f' {query} \n A:'
    for k, v in msg.items():
        m['role'], m['content'] = k, v
    return [m]

def recommend_result(query):
    user_msg_history = []
    msg_prompt_init = {
        'recom' : {
                    'system' : "당신은 사용자의 질문에 따라 은행 이벤트를 추천하는 유용한 도우미입니다.", 
                    'user' : "'물론이죠!'로 시작하는 간단한 인사말을 한 문장씩 작성해 보세요. 사용자에게 은행 이벤트 항목을 추천합니다.", 
                  },
        'desc' : {
                    'system' : "당신은 친절하게 답변해주시는 유용한 도우미입니다.", 
                    'user' : "사용자에게 해당 항목을 설명하기 위해 '물론이죠!'로 시작하는 간단한 인사말을 작성해주세요.", 
                  },
        'intent' : {
                    'system' : "당신은 사용자의 질문 의도를 이해하는 유용한 도우미입니다.",
                    'user' : "다음 문장은 '설명', '추천', '검색' 중 어떤 카테고리에 속합니까? 카테고리만 표시합니다. \n content:"
                    }
    }


    # 1. 사용자의 의도를 파악
    user_intent = set_prompt('intent', query, msg_prompt_init)
    user_intent = get_chatgpt_msg(user_intent)
    #print("user_intent : ", user_intent)

    # 2. 사용자의 쿼리에 따라 prompt 생성    
    intent_data = set_prompt(user_intent, query, msg_prompt_init)
    intent_data_msg = get_chatgpt_msg(intent_data).replace("\n", "").strip()
    #print("intent_data_msg : ", intent_data_msg)

    # 3. 추천 또는 검색이면
    if ('추천' in user_intent) or ('검색' in user_intent):
        recom_msg = str()
        # 기존에 메세지가 있으면 쿼리로 대체
        if (len(user_msg_history) > 0 ) and (user_msg_history[-1]['role'] == 'assistant'):
            query = user_msg_history[-1]['content']['feature']
            
        # 유사 아이템 top_k = 3개 가져오기
        top_result = get_query_sim_top_k(query, top_k=3)

        # 검색이면, 자기 자신의 컨텐츠는 제외
        top_index = top_result[1].numpy() #if '추천' in user_intent else top_result[1].numpy()[1:]

        # '번호','제목', '이벤트 기간', 'img', 'img_text'를 가져와서 출력
        r_set_d = df.iloc[top_index, :][['번호','제목', '이벤트 기간', '알아두세요', 'img', 'img_text']]
        r_set_d = r_set_d.rename(columns = {'번호':'event_id', '제목':'event_title', '이벤트 기간':'event_period','img':'event_img','알아두세요':'text'})
        r_set_d = json.loads(r_set_d.to_json(orient="records"))
        for r in r_set_d:
            for _, v in r.items():
                recom_msg += f"{v} \n"
            recom_msg += "\n"
        user_msg_history.append({'role' : 'assistant', 'content' : f"{intent_data_msg} {str(recom_msg)}"})
        #print(f"\n recom data : {intent_data_msg} {str(recom_msg)}")
        return r_set_d #recom_msg

# 특정 고객의 id로 고객 정보 호출
selected_id = random.randint(0, 1000)
selected_customer = customer_dict.get(selected_id)
query = f"""아래 고객 정보 (ID: {selected_id}), {selected_customer} 를 기반으로 적절한 금융 이벤트를 추천해 주세요."""
#query = f"""아래 고객 정보 (ID: {selected_id})를 기반으로 적절한 금융 이벤트를 추천해 주세요.
#{selected_customer}
#"""

#고객 정보
print(query)

#이벤트 추천 결과
answer = recommend_result(query)
print(answer)

아래 고객 정보 (ID: 151), {'id': 151, 'Top3_소비_키워드': ['가정생활/서비스', '의료', '소형유통점'], '직업군': '무직/기타', '생년월일': 1947, '성별': '남'} 를 기반으로 적절한 금융 이벤트를 추천해 주세요.
torch.return_types.topk(
values=tensor([0.8516, 0.8489, 0.8475]),
indices=tensor([630, 543, 487]))
[{'event_id': 756, 'event_title': '연금저축펀드 「연말정산 환급원정대」 고객 이벤트', 'event_period': '2020.11.05 ~ 2020.12.31', 'text': '집합투자상품은 예금자보호법에 따라 예금보험공사가 보호하지 않습니다.집합투자기구는 운용결과에 따라 투자원금의 손실이 발생할 수 있으며 그 손실은 투자자에게 귀속됩니다.펀드가입을 결정하시기 전에 투자대상, 투자 위험, 환매방법 및 보수 등에 관하여 간이 투자설명서 또는 투자설명서를 반드시 읽어보시기 바랍니다.해외의 증권에 투자하는 펀드는 투자 대상 국가의 시장, 경제상황 및 환율의 가치가 변동되어 손실이 발생할 수 있습니다.과세기준 및 과세방법은 향후 세법 개정 등에 따라 변동될 수 있습니다.연금수령요건을 갖추지 못하고 중도해지 하거나 계약기간 종료 후 연금 이외의 형태로 수령하는 경우 세액공제 받은 납입원금 및 수익에 대해 기타소득세(16.5%)세율이 부과될 수 있습니다.경품 수령을 위한 고객정보상의 연락처(휴대폰 번호)를 확인하여 주시기 바랍니다. 당첨자 발표일 이후 한 달 이내 경품 수령거부 또는 연락처 오류 등의 사유로 연락 불가 시 당첨이 무효처리 됩니다.본 이벤트는 KB국민은행 및 제휴업체의 사정에 따라 변경 또는 중단될 수 있습니다. (단, 기조건 충족고객 및 당첨고객 제외)경품제공을 위해 수탁업체에게 고객님의 정보를 제공합니다.(수탁업체:㈜코원비젼, 제공범위:전화번호, 대상업무:경품지급)기타 자세한 이벤트 내용은 창구

/var/folders/qw/4pc1z8g93n746cysqbq8tq840000gn/T/ipykernel_5131/1574581155.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['score'] = top_result_score
